In [1]:
!pip3 install torch torchvision

    100% |████████████████████████████████| 519.5MB 33kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x590cc000 @  0x7f34ba9261c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 19.2MB/s 
    100% |████████████████████████████████| 2.0MB 2.4MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [ ]:
!mkdir data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-testing.data -P ./data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-training-true.data -P ./data
# !ls data

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

import pandas as pd


In [0]:
class NeuralNetwork(nn.Module):

    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.fc1 = nn.Linear(10, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 100)
#         self.fc4 = nn.Linear(100, 100)
#         self.fc4_drop = nn.Dropout(p=0.2)
#         self.fc4 = nn.Linear(50, 50)
#         self.fc5 = nn.Linear(50, 50)

        self.fc5 = nn.Linear(100, 10)

    def forward(self, x):
        x = nn.ELU()(self.fc1(x))
        x = nn.ELU()(self.fc2(x))
        x = nn.ELU()(self.fc3(x))
#         x = self.fc4(x)
#         x = nn.ELU()(self.fc4_drop(x))
#         x = nn.ELU()(self.fc4(x))
#         x = nn.ELU()(self.fc5(x))
        
        x = F.log_softmax(self.fc5(x))
        return x;

In [0]:
device = torch.device('cuda')

In [0]:
def gen_confusion_matrix(output, labels):
  _, output = torch.max(output.data, 1)
  
  y_actu = pd.Categorical(labels, categories=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
  y_pred = pd.Categorical(output, categories=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
  df_confusion = pd.crosstab(y_actu, y_pred, rownames=['Label'], colnames=['Predicted'])
  
  return df_confusion

def get_metrics(output, labels):
  _, output = torch.max(output.data, 1)
  
  accuracy_s = accuracy_score(labels, output)
  f1_s = f1_score(labels, output, average="macro")
  precision_s = precision_score(labels, output, average="macro")
  recall_s = recall_score(labels, output, average="macro")
  
#   return score(labels, output)
  return (accuracy_s, f1_s, precision_s, recall_s)

def get_classification_report(output, labels):
  _, output = torch.max(output.data, 1)
  target_names = '0,1,2,3,4,5,6,7,8,9'.split(',')#convert to array of strings
  
  return classification_report(labels, output, target_names=target_names)

def new_model():
  model = NeuralNetwork().to(device)
  lr = 0.01
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  
  return model, optimizer

In [23]:
data_t = pd.read_csv('data/poker-hand-training-true.data', names=['S1', 'C1', 'S2', 'C2', 'S3', 'C3', 'S4', 'C4', 'S5', 'C5', 'CLASS'])
# data_t[data_t['CLASS'] == 5 ]
data_t.groupby('CLASS').count()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5
CLASS,,,,,,,,,,
0,12493,12493,12493,12493,12493,12493,12493,12493,12493,12493
1,10599,10599,10599,10599,10599,10599,10599,10599,10599,10599
2,1206,1206,1206,1206,1206,1206,1206,1206,1206,1206
3,513,513,513,513,513,513,513,513,513,513
4,93,93,93,93,93,93,93,93,93,93
5,54,54,54,54,54,54,54,54,54,54
6,36,36,36,36,36,36,36,36,36,36
7,6,6,6,6,6,6,6,6,6,6
8,5,5,5,5,5,5,5,5,5,5


In [0]:
data = pd.read_csv('data/poker-hand-testing.data', header=None).values # bigger file (1 MIL)
# data = pd.read_csv('data/poker-hand-training-true.data', header=None).values
data = np.array(data)
# data[:, -2:-1]
x = data[:, 0:10]
# y = data[:, 10:11]
y = [d[-1] for d in data]

x_tensor = torch.tensor(x, dtype=torch.float).to(device)#
y_tensor = torch.tensor(y, dtype=torch.long).to(device)#

In [26]:
epochs = 2000
model, optimizer = new_model()
for epoch in range(epochs):
#   x = np.array(x)
#   np.random.shuffle(data)
#   x_tensor = torch.tensor(x, dtype=torch.float).cuda()
#   y_tensor = torch.tensor(y, dtype=torch.long).cuda()
  
  output = model(x_tensor)
  loss = F.nll_loss(output, y_tensor)
  if(epoch%100 == 0):
    print('epoch: {}, loss: {}'.format(epoch, loss))
    
  optimizer.zero_grad()
  
  loss.backward()
  
#   for param in model.parameters():
#     print(param.grad)
  
  optimizer.step()

output = model(x_tensor)

# print(gen_confusion_matrix(output, y_tensor))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch: 0, loss: 2.4617972373962402
epoch: 100, loss: 0.9198356866836548
epoch: 200, loss: 0.9070908427238464
epoch: 300, loss: 0.8366517424583435
epoch: 400, loss: 0.7421240210533142
epoch: 500, loss: 0.6395681500434875
epoch: 600, loss: 0.5783801674842834
epoch: 700, loss: 0.423604279756546
epoch: 800, loss: 0.29147133231163025
epoch: 900, loss: 0.20513126254081726
epoch: 1000, loss: 0.1684514880180359
epoch: 1100, loss: 0.07387106865644455
epoch: 1200, loss: 0.028265437111258507
epoch: 1300, loss: 0.012535015121102333
epoch: 1400, loss: 0.007029028609395027
epoch: 1500, loss: 0.004489030223339796
epoch: 1600, loss: 0.0031315810047090054
epoch: 1700, loss: 0.002315098885446787
epoch: 1800, loss: 0.00178162578959018
epoch: 1900, loss: 0.001414202037267387


In [31]:
output = model(x_tensor)

print(gen_confusion_matrix(output, y_tensor))
accuracy_s, f1_s, precision_s, recall_s = get_metrics(output, y_tensor)
print("accuracy: {}, f1_score: {}, precision: {}, recall: {}".format(accuracy_s, f1_s, precision_s, recall_s))

print(get_classification_report(output, y_tensor))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Predicted       0       1      2      3     4     5    6    7   8   9
Label                                                                
0          499248     746      0      0   123  1073    0    0   0  19
1             378  420952    615     74   145   233    0   61  18  22
2               0    2422  43473   1560     0     0   24  143   0   0
3               0      88   2176  18425     0     0  121  311   0   0
4             643     107      0      0  3035     2    0    0  80  18
5            1927       4      0      0     2    63    0    0   0   0
6               0       0    258    339     0     0  826    1   0   0
7               0       0      1    178     0     0   35   16   0   0
8               2       0      0      0    10     0    0    0   0   0
9               2       1      0      0     0     0    0    0   0   0
accuracy: 0.986038, f1_score: 0.5398897999742079, precision: 0.5628722269706289, recall: 0.5240053506561343
             precision    recall  f1-score   support

In [22]:
epochs = 2000
k = 5
kfold = KFold(n_splits=k)

#metrics
mean_accuracy = 0
mean_f1_score = 0
mean_precision_score = 0
mean_recall_score = 0

# print(data)
for train_index, test_index in kfold.split(data):
  model, optimizer = new_model()

  print('FOLD')
  X_train, X_test = data[train_index], data[test_index]
  
  x_train = X_train[:, 0:10]
  y_train = [d[-1] for d in X_train]
  
  for epoch in range(epochs):

    x_train_tensor = torch.tensor(x_train, dtype=torch.float).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
  
    output = model(x_train_tensor)
    loss = F.nll_loss(output, y_train_tensor)
    if(epoch%100 == 0):
      print('epoch: {}, loss: {}'.format(epoch, loss))
    optimizer.zero_grad()

    loss.backward()
    optimizer.step()
  
  x_test = X_test[:, 0:10]
  y_test = [d[-1] for d in X_test]
  
  x_test_tensor = torch.tensor(x_test, dtype=torch.float).to(device)
  y_test_tensor = torch.tensor(y_test, dtype=torch.float).to(device)
  
  output = model(x_test_tensor)
#   print(output.shape)
  print('confusion matrix:')
  print(gen_confusion_matrix(output, y_test_tensor))
  
  ac, f1_s, precision_s, recall_s = get_metrics(output, y_test_tensor)
  print('accuracy: {}, f1_score: {}, precision: {}, recall: {} '.format(ac, f1_s, precision_s, recall_s))
  
#   print('classification report:')
#   print(get_classification_report(output, y_test_tensor))
  
  mean_accuracy += ac
  mean_f1_score += f1_s
  mean_precision_score += precision_s
  mean_recall_score += recall_s

print('mean accuracy: {}, mean f1_score: {}, mean precision: {}, mean recall: {}'
      .format(mean_accuracy/k, mean_f1_score/k, mean_precision_score/k, mean_recall_score/k))
#   torch.max(output[0], 0)

FOLD
epoch: 0, loss: 2.171663999557495


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch: 100, loss: 0.9234766960144043
epoch: 200, loss: 0.9008291959762573
epoch: 300, loss: 0.8266838788986206
epoch: 400, loss: 0.730699360370636
epoch: 500, loss: 0.6029488444328308
epoch: 600, loss: 0.517005980014801
epoch: 700, loss: 0.4061794579029083
epoch: 800, loss: 0.40509364008903503
epoch: 900, loss: 0.2486802637577057
epoch: 1000, loss: 0.29530465602874756
epoch: 1100, loss: 0.29750820994377136
epoch: 1200, loss: 0.1258937567472458
epoch: 1300, loss: 1.284822940826416
epoch: 1400, loss: 0.187806636095047
epoch: 1500, loss: 0.10241521149873734
epoch: 1600, loss: 1.5873463153839111
epoch: 1700, loss: 0.15035714209079742
epoch: 1800, loss: 0.07248293608427048
epoch: 1900, loss: 0.03803427517414093
confusion matrix:
Predicted     0     1    2   3  4  5  6  7  8  9
Label                                           
0          2401    52    0   0  0  1  0  0  0  0
1            50  2099   12   1  8  1  0  0  0  0
2             0    39  175   8  0  0  1  0  0  0
3             0     3

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


epoch: 100, loss: 0.9268632531166077
epoch: 200, loss: 0.8843262791633606
epoch: 300, loss: 0.8273105025291443
epoch: 400, loss: 0.7285082936286926
epoch: 500, loss: 0.627004086971283
epoch: 600, loss: 0.49006763100624084
epoch: 700, loss: 0.43600064516067505
epoch: 800, loss: 0.3136351704597473
epoch: 900, loss: 0.24408593773841858
epoch: 1000, loss: 0.15926189720630646
epoch: 1100, loss: 0.1652134656906128
epoch: 1200, loss: 0.08518611639738083
epoch: 1300, loss: 0.16901615262031555
epoch: 1400, loss: 0.07533179223537445
epoch: 1500, loss: 0.0451594702899456
epoch: 1600, loss: 0.02710624784231186
epoch: 1700, loss: 0.01635773479938507
epoch: 1800, loss: 0.010262589901685715
epoch: 1900, loss: 0.006895275786519051
confusion matrix:
Predicted     0     1    2   3   4  5  6  7  8  9
Label                                            
0          2433    50    0   0   0  9  0  0  0  0
1            62  2033   11   2   1  2  0  0  0  0
2             0    46  200  19   0  0  0  0  0  0
3      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


epoch: 100, loss: 0.9191012978553772
epoch: 200, loss: 0.8926832675933838
epoch: 300, loss: 0.8307335376739502
epoch: 400, loss: 0.6875672936439514
epoch: 500, loss: 0.6368933916091919
epoch: 600, loss: 0.3310507833957672
epoch: 700, loss: 0.1442742645740509
epoch: 800, loss: 0.3453422486782074
epoch: 900, loss: 3.0297482013702393
epoch: 1000, loss: 0.2353421449661255
epoch: 1100, loss: 0.06056586280465126
epoch: 1200, loss: 0.020925205200910568
epoch: 1300, loss: 0.01203705184161663
epoch: 1400, loss: 0.00799144059419632
epoch: 1500, loss: 0.005643028765916824
epoch: 1600, loss: 0.004147931467741728
epoch: 1700, loss: 0.0031209064181894064
epoch: 1800, loss: 0.0023959921672940254
epoch: 1900, loss: 0.0018853121437132359
confusion matrix:
Predicted     0     1    2    3   4  5  6  7  8  9
Label                                             
0          2477     0    0    0   1  5  0  0  0  0
1             0  2114    0    0   3  1  0  0  0  0
2             0     1  240    2   0  0  0  0  0

In [169]:
x = torch.tensor([2,12,1,6,4,5,1,12,3,12], dtype=torch.float).cuda()
output = model(x)
torch.max(output[0], 0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


(tensor(-177.0979, device='cuda:0', grad_fn=<MaxBackward0>),
 tensor(0, device='cuda:0'))